In [1]:
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from core.elements.dormitory import Dormitory
from core.elements.student import Student
from core.elements.floor import Floor
from core.elements.room import Room

from core.engine.state_processor import StateProcessor

from frontend.pygame_manager import PygameManager

In [ ]:
dormitory = Dormitory()
dormitory.select_room(0)
state_processor = StateProcessor(dormitory)

pygame_manager = PygameManager()

# Main game loop
clock = pygame.time.Clock()
tick = 30
running = True
paused = True
manual = False
time = 24

is_exam_season = False
exams_ended = False

pygame_manager.draw(dormitory, time, state_processor.exam_difficulty)
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                if manual:
                    state_processor.iterate(is_exam_season)
                    time += 1
                paused = not paused
            else:
                running = pygame_manager.handle_keydown(event, dormitory)
    if not paused and not manual:
        state_processor.iterate(is_exam_season)
        time += 1

    if exams_ended and not is_exam_season:
        exams_ended = False
        state_processor.setup_new_exam()
        dormitory.finish_exam_session()

    exams_ended = is_exam_season
    pygame_manager.draw(dormitory, time, state_processor.exam_difficulty)
    is_exam_season = (time // 24) % 7 == 0
    clock.tick(tick)
    

pygame.quit()

Student 0.8029968158042489 created
Student 0.6416645158718153 created
Student 0.8004535452340019 created
Student 0.47285054515531966 created
Student 0.9145508592749052 created
Student 0.11303735173959029 created
Student 0.23649110472159857 created
Student 0.40447252034413905 created
Student 0.4826751992948275 created
Student 0.6131018669965763 created
Student 0.05 created
Student 0.3821562613826559 created
Student 0.7899404312278417 created
Student 0.40635879413884995 created
Student 0.7855738281854048 created
Student 0.40069679638450384 created
Student 0.34781998588533936 created
Student 0.403084888306245 created
Student 0.7239962936467336 created
Student 0.4942543945940704 created
Student 0.2518953804690801 created
Student 0.12174070778492796 created
Student 0.5613325956798565 created
Student 0.18845331635681217 created
Student 0.30878308899416607 created
Student 0.5160273834600592 created
Student 0.6717290460637515 created
Student 0.6779193088583362 created
Student 0.755102760279586